# Анализ тональности отзывов на фильмы: настройка параметров

*В этом задании вам предстоит поэкспериментировать с параметрами вашей модели для сентимент-анализа. Все задания выполняются на том же датасете, что и на прошлой неделе.*

## Загрузка библиотек

In [9]:
import nltk
from nltk.corpus import stopwords, movie_reviews
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

## Загрузка и подготовка данных

In [10]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/sergio/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [11]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [12]:
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]

texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

## Вспомогательные функции

In [21]:
def make_pipeline(vectorizer, classifier):
    return Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', classifier)
        ])

## Задание №1
*Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество ( .mean() ) и стандартное отклонение ( .std() ) по fold'ам для: а) pipeline из CountVectorizer() и LogisticRegression(), б) pipeline из TfidfVectorizer() и LogisticRegression(). В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б*

In [22]:
pipeline_1_a = make_pipeline(CountVectorizer(), LogisticRegression(solver='liblinear'))
score_1_a = cross_val_score(pipeline_1_a, texts, labels, cv=5)

In [23]:
print(f"Accuracy на кросс-валидации - {score_1_a.mean()} +- {score_1_a.std()}")

Accuracy на кросс-валидации - 0.841 +- 0.01677796173556255


In [25]:
pipeline_1_b = make_pipeline(TfidfVectorizer(), LogisticRegression(solver='liblinear'))
score_1_b = cross_val_score(pipeline_1_b, texts, labels, cv=5)

In [26]:
print(f"Accuracy на кросс-валидации - {score_1_b.mean()} +- {score_1_b.std()}")

Accuracy на кросс-валидации - 0.8210000000000001 +- 0.004062019202317978


In [32]:
ans1 = ' '.join(map(str, [score_1_a.mean(), score_1_a.std(), score_1_b.mean(), score_1_b.std()]))

In [33]:
with open('ans1.txt', 'w') as f:
    f.write(ans1)

## Задание №2
*Попробуйте задавать разные значения параметра min_df у CountVectorizer. Оцените качество вашего классификатора с min_df=10 и с min_df=50.*

In [61]:
pipeline_2_a = make_pipeline(CountVectorizer(min_df=10), LogisticRegression(solver='liblinear'))
score_2_a = cross_val_score(pipeline_2_a, texts, labels, cv=5).mean()

In [62]:
print(f"Accuracy на кросс-валидации - {score_2_a}")

Accuracy на кросс-валидации - 0.8390000000000001


In [63]:
pipeline_2_b = make_pipeline(CountVectorizer(min_df=50), LogisticRegression(solver='liblinear'))
score_2_b = cross_val_score(pipeline_2_b, texts, labels, cv=5).mean()

In [68]:
print(f"Accuracy на кросс-валидации - {score_2_b}")

Accuracy на кросс-валидации - 0.813


In [65]:
ans2 = ' '.join(map(str, [score_2_a, score_2_b]))

In [66]:
with open('ans2.txt', 'w') as f:
    f.write(ans2)

## Задание №3
*Попробуйте использовать разные классификаторы после CountVectorizer. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.*

In [67]:
pipeline_3_a = make_pipeline(CountVectorizer(), LinearSVC())
score_3_a = cross_val_score(pipeline_3_a, texts, labels, cv=5).mean()

/home/sergio/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [69]:
print(f"Accuracy на кросс-валидации - {score_3_a}")

Accuracy на кросс-валидации - 0.8325000000000001


In [73]:
pipeline_3_b = make_pipeline(CountVectorizer(), SGDClassifier())
score_3_b = cross_val_score(pipeline_3_b, texts, labels, cv=5).mean()

/home/sergio/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [74]:
print(f"Accuracy на кросс-валидации - {score_3_b}")

Accuracy на кросс-валидации - 0.7685000000000001


In [75]:
ans3 = str(min([score_1_a.mean(), score_3_a, score_3_b]))

In [76]:
with open('ans3.txt', 'w') as f:
    f.write(ans3)

## Задание №4
*Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.*

In [53]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/sergio/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [55]:
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [77]:
pipeline_4_a = make_pipeline(CountVectorizer(stop_words=stop_words, analyzer='word'), LogisticRegression(solver='liblinear'))
score_4_a = cross_val_score(pipeline_4_a, texts, labels, cv=5).mean()

In [78]:
print(f"Accuracy на кросс-валидации - {score_4_a}")

Accuracy на кросс-валидации - 0.841


In [79]:
pipeline_4_b = make_pipeline(CountVectorizer(stop_words='english'), LogisticRegression(solver='liblinear'))
score_4_b = cross_val_score(pipeline_4_b, texts, labels, cv=5).mean()

In [80]:
print(f"Accuracy на кросс-валидации - {score_4_b}")

Accuracy на кросс-валидации - 0.8390000000000001


In [81]:
ans4 = ' '.join(map(str, [score_4_a, score_4_b]))

In [82]:
with open('ans4.txt', 'w') as f:
    f.write(ans4)

## Задание №5
*Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.*

In [83]:
pipeline_5_a = make_pipeline(CountVectorizer(ngram_range=(1, 2)), LogisticRegression(solver='liblinear'))
score_5_a = cross_val_score(pipeline_5_a, texts, labels, cv=5).mean()

In [84]:
print(f"Accuracy на кросс-валидации - {score_5_a}")

Accuracy на кросс-валидации - 0.8525


In [85]:
pipeline_5_b = make_pipeline(CountVectorizer(analyzer='char_wb', ngram_range=(3, 5)), LogisticRegression(solver='liblinear'))
score_5_b = cross_val_score(pipeline_5_b, texts, labels, cv=5).mean()

In [86]:
print(f"Accuracy на кросс-валидации - {score_5_b}")

Accuracy на кросс-валидации - 0.82


In [87]:
ans5 = ' '.join(map(str, [score_5_a, score_5_b]))

In [88]:
with open('ans5.txt', 'w') as f:
    f.write(ans5)